In [53]:
%pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [54]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import calendar
import datetime
import requests
from bs4 import BeautifulSoup
import traceback

In [55]:
path = os.path.join(os.getcwd(),"Data")
revenuepath = os.path.join(path,"Revenue")

In [56]:
# def download_revenue_profit(code,name): 

#     def create_driver():
#         chromeOptions = webdriver.ChromeOptions()
#         chromeOptions.add_argument("--headless")
#         chromeOptions.add_experimental_option("prefs",{"download.default_directory":revenuepath})
#         driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
#         return driver

#     def download():
#         columns = ["security code","security name",'total income','net sales/revenue from operations','expenditure','net profit','basic eps for continuing operation','basic & diluted eps before extraordinary items',"year","quartile"]
#         code_df = pd.DataFrame(columns=columns)
#         for q in range(55,104):
#             url = "https://www.bseindia.com/corporates/results.aspx?Code=" + str(code) +"&Company="+ str(name) +"&qtr="+ str(q) +"&RType=D"
#             driver.get(url)
# #             time.sleep(1)
#             html = driver.page_source
#             soup = BeautifulSoup(html, "html")

#             table = soup.find_all("table",attrs={"id":"ContentPlaceHolder1_tbl_typeID"})
#             table = pd.read_html(str(table))[0]
#             table = table[[0,1]]
#             table.dropna(inplace=True)
#             table = table.transpose()
#             table.columns = table.iloc[0]
#             table = table[1:]
#             table.columns = map(str.lower, table.columns)
#             table.drop(["description"],inplace=True,axis=1)
#             try:
#                 table["date begin"] = pd.to_datetime(table["date begin"])
#                 date = table.iloc[0]["date begin"]
#                 table["quartile"] =  (date.month - 1) // 3 + 1
#                 table["year"] = date.year
#                 table["security name"] = name
#                 table["security code"] = code
#                 code_df = code_df.append(table,ignore_index=True)
#             except Exception as e:
#                 pass

            
#         code_df["eps"] = code_df['basic eps for continuing operation'].fillna('') + code_df['basic & diluted eps before extraordinary items'].fillna('')
#         keep_columns = ['security code', 'security name', 'total income','net sales/revenue from operations', 'expenditure', 'net profit','year', 'quartile','eps']
#         res = ["other operating income","other income","net sales / income from operations","total income"]
#         code_df[res] = code_df[res].apply(pd.to_numeric)
#         code_df = code_df.fillna(0)
#         code_df["total income"] = code_df["total income"] + code_df["other operating income"] + code_df["other income"] + code_df["net sales / income from operations"]
#         code_df = code_df[keep_columns]
#         code_df.to_csv(os.path.join(revenuepath,str(code)+".csv"),index=None)
#     driver = create_driver()
#     download()
#     driver.quit()

In [57]:
# download_revenue_profit("500002","ABB India Limited")

In [58]:
def download_revenue_profit(code,name): 

    path = os.path.join(os.getcwd(),"Data\\Revenue")

    def create_driver():
        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    def download():
        columns = ["security code","security name",'revenue','income','expenditure','profit','eps',"year","quartile"]
        code_df = pd.DataFrame(columns=columns)
        for q in range(55,108):
            url = "https://www.bseindia.com/corporates/results.aspx?Code=" + str(code) +"&Company="+ str(name) +"&qtr="+ str(q) +"&RType=D"
            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html, "html")

            table = soup.find_all("table",attrs={"id":"ContentPlaceHolder1_tbl_typeID"})
            table = pd.read_html(str(table))[0]
            table = table[[0,1]]
            table.dropna(inplace=True)
            table = table.transpose()
            table.columns = table.iloc[0]
            table = table[1:]
            table.columns = map(str.lower, table.columns)
            table.drop(["description"],inplace=True,axis=1)
            try:
                table["date begin"] = pd.to_datetime(table["date begin"])
                date = table.iloc[0]["date begin"]
                table["quartile"] =  (date.month-1)//3 + 1
                table["year"] = date.year
                table["security name"] = name
                table["security code"] = code
                cols = table.columns
                mycols = ['revenue','income','expenditure','profit','eps']
                row = {}
                row["security name"] = name
                row["security code"] = code
                row["year"] = date.year
                row["quartile"] = (date.month-1)//3 + 1
                for my in mycols:
                    res = [c for c in cols if my in c]
                    if my == "income":
                        p = [c for c in res if "total income" == c ] 
                        res = p or res
                    elif my == "profit":
                        p = [c for c in res if "net profit" == c]
                        res = p or res
                    elif my == "expenditure":
                        p = [c for c in cols if "expenses" in c]
                        res = p or res
                    elif my =="eps":
                        a = "Basic for discontinued & continuing operation"
                        b = "Diluted for discontinued & continuing operation"
                        p = [c for c in cols if a.lower() in c or b.lower() in c]
                        res = p or res
                    elif my == "revenue":
                        p = [c for c in cols if "sales" in c]
                        res = p or res
                        pass
                    row[my] = table[res].values[0][0]
                code_df = code_df.append(row,ignore_index=True)
            except Exception as e:
                traceback.print_exc()
        code_df.to_csv(os.path.join(path,str(code)+".csv"),index=None)
    driver = create_driver()
    download()
    driver.quit()

In [59]:
# download_revenue_profit("532540","TATA CONSULTANCY SERVICES LTD")

In [60]:
df_equity = pd.read_csv("C:\\Users\\venu\\Desktop\\Stock Market Analysis\\Data\\Equity.csv")
security_numbers = df_equity["Security Code"].tolist()
security_names = df_equity["Security Name"].tolist()
sec = list(zip(security_numbers, security_names))

In [61]:
# df_equity

In [62]:
# security_numbers[279 : 299]

In [63]:
%%time

for sec_num, sec_name in sec[279 : 299]:
    print(sec_num, sec_name)
    download_revenue_profit(sec_num, sec_name)
#     break

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


501242 TCI FINANCE LTD.




[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
Traceback (most recent call last):
  File "C:\Users\venu\anaconda3\envs\tf\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'date begin'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 31, in download
    table["date begin"] = pd.to_datetime(table["date begin"])
  File "C:\Users\venu\anaconda3\envs\tf\lib\site-pac

Traceback (most recent call last):
  File "C:\Users\venu\anaconda3\envs\tf\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'date begin'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 31, in download
    table["date begin"] = pd.to_datetime(table["date begin"])
  File "C:\Users\venu\anaconda3\envs\tf\lib\site-packages\pandas\core\frame.py", line 3024, in __getitem__
    indexer = self.columns.get_loc(key)
  File "C:\Us

501261 KRATOS ENERGY & INFRASTRUCTURE LTD.




...

[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
Traceback (most recent call last):
  File "C:\Users\venu\anaconda3\envs\tf\lib\site-packages\pandas\core\indexes\base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'date begin'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 31, in download
    table["date begin"] = pd.to_datetime(table["date begin"])
  File "C:\Users\venu\anaconda3\envs\tf\lib\site-pac

501622 AMALGAMATED ELECTRICITY CO.LTD.




[WDM] - Driver [C:\Users\venu\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my

Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most recent call last):
  File "<ipython-input-58-5beadee9cbf9>", line 64, in download
    row[my] = table[res].values[0][0]
IndexError: index 0 is out of bounds for axis 0 with size 0
Traceback (most rece

Wall time: 21min 38s
